## Visualise stokes annulus model (flow etc)

In [1]:
# to fix trame issue
import nest_asyncio
nest_asyncio.apply()
import os

In [2]:
import petsc4py
import underworld3 as uw
import numpy as np
import sympy

[esdhcp-137.anu.edu.au:15366] shmem: mmap: an error occurred while determining whether or not /var/folders/tx/95gr762j29z4tt5d1dnqlgth0000gn/T//ompi.esdhcp-137.501/jf.0/2320433152/sm_segment.esdhcp-137.501.8a4f0000.0 could be created.


In [3]:
ls -trl output/cartesian/ | tail

total 0
drwxr-xr-x@ 1409 lmoresi  staff  45088 Aug 19 21:32 Ra1e6.0/
drwxr-xr-x  1585 lmoresi  staff  50720 Aug 20 07:40 Ra1e7.0/
drwxr-xr-x@ 1208 lmoresi  staff  38656 Aug 20 10:19 Ra1e5.0/
drwxr-xr-x@  608 lmoresi  staff  19456 Aug 20 12:41 Ra1e4.0/


In [4]:
ls -trl output/cartesian/Ra1e4.0/ | tail

-rw-r--r--@   1 lmoresi  staff  490280 Aug 20 12:40 cartesian_Ra1e4.0_res15.mesh.T.00016.h5
-rw-r--r--@   1 lmoresi  staff    3086 Aug 20 12:40 cartesian_Ra1e4.0_res15.mesh.00016.xdmf
-rw-r--r--@   1 lmoresi  staff  280632 Aug 20 12:40 cartesian_Ra1e4.0_res15.mesh.U.00017.h5
-rw-r--r--@   1 lmoresi  staff   95560 Aug 20 12:40 cartesian_Ra1e4.0_res15.mesh.P.00017.h5
-rw-r--r--@   1 lmoresi  staff  490280 Aug 20 12:40 cartesian_Ra1e4.0_res15.mesh.T.00017.h5
-rw-r--r--@   1 lmoresi  staff    3086 Aug 20 12:40 cartesian_Ra1e4.0_res15.mesh.00017.xdmf
-rw-r--r--@   1 lmoresi  staff  280632 Aug 20 12:41 cartesian_Ra1e4.0_res15.mesh.U.00018.h5
-rw-r--r--@   1 lmoresi  staff   95560 Aug 20 12:41 cartesian_Ra1e4.0_res15.mesh.P.00018.h5
-rw-r--r--@   1 lmoresi  staff  490280 Aug 20 12:41 cartesian_Ra1e4.0_res15.mesh.T.00018.h5
-rw-r--r--@   1 lmoresi  staff    3086 Aug 20 12:41 cartesian_Ra1e4.0_res15.mesh.00018.xdmf


In [5]:
# This is a bit repetitive, sorry 

checkpoint_dir = f"output/cartesian/Ra1e4.0"
name = "cartesian_Ra1e4.0"
resolution = 15


checkpoint_base = f"{name}_res{resolution}"
meshfile = os.path.join(checkpoint_dir, checkpoint_base) + ".mesh.00000.h5"
image_dir = os.path.join(checkpoint_dir,"images")
os.makedirs(image_dir, exist_ok=True)

In [6]:
mesh = uw.discretisation.Mesh(meshfile, 
                              coordinate_system_type=uw.coordinates.CoordinateSystemType.CARTESIAN)
    
x = mesh.N.x
y = mesh.N.y

v_soln = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
t_soln = uw.discretisation.MeshVariable("T", mesh, 1, degree=3)
flux = uw.discretisation.MeshVariable(r"dTdz", mesh, 1, degree=1)

In [18]:
steps = range(0,100,1)

In [19]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pl = pv.Plotter(window_size=[1000, 500])

for step in steps:

    try:
        v_soln.read_timestep(checkpoint_base, "U", step, outputPath=checkpoint_dir)
        t_soln.read_timestep(checkpoint_base, "T", step, outputPath=checkpoint_dir)
        print(f"Plotting step {step}")

    except:
        print(f"Failed to read step {step} correctly")
        continue

    pl.clear()

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    # pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym[0])
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln, alpha=None)
    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln, alpha=None)

    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym)
    
    # point sources at cell centres
    skip = 10
    points = np.zeros((mesh._centroids[::skip].shape[0], 3))
    points[:, 0] = mesh._centroids[::skip, 0]
    points[:, 1] = mesh._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)

    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Black",
        edge_opacity=0.1,
        show_edges=True,
        scalars="T",
        use_transparency=False,
        opacity=1.0,
        clim=[0.0,1.0],
        # show_scalar_bar=False
    )

    pvstream = pvmesh.streamlines_from_source(
        point_cloud,
        vectors="V",
        integration_direction="both",
        max_time=1,
        surface_streamlines=True,
    )
    

    pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)    

    pl.camera_position = [(3.0, 0.5, 6.750994846396983),
                          (3.0, 0.5, 0.0),
                          (0.0, 1.0, 0.0)]

    imagefile = os.path.join(image_dir, checkpoint_base) + f"_V_T_{step}.png"
    
    pl.screenshot(filename=imagefile,
                  window_size=(1000, 500),
                  return_img=False)



Plotting step 0
Plotting step 1
Plotting step 2
Plotting step 3
Plotting step 4
Plotting step 5
Plotting step 6
Plotting step 7
Plotting step 8
Plotting step 9
Plotting step 10
Plotting step 11
Plotting step 12
Plotting step 13
Plotting step 14
Plotting step 15
Plotting step 16
Plotting step 17
Plotting step 18
Plotting step 19
Plotting step 20
Plotting step 21
Plotting step 22
Plotting step 23
Plotting step 24
Plotting step 25
Plotting step 26
Plotting step 27
Plotting step 28
Plotting step 29
Plotting step 30
Plotting step 31
Plotting step 32
Plotting step 33
Plotting step 34
Plotting step 35
Plotting step 36
Plotting step 37
Plotting step 38
Plotting step 39
Plotting step 40
Plotting step 41
Plotting step 42
Plotting step 43
Plotting step 44
Plotting step 45
Plotting step 46
Plotting step 47
Plotting step 48
Plotting step 49
Plotting step 50
Plotting step 51
Plotting step 52
Plotting step 53
Plotting step 54
Plotting step 55
Plotting step 56
Plotting step 57
Plotting step 58
Plottin

In [17]:
pl.show(jupyter_backend="trame")


Widget(value='<iframe src="http://localhost:59691/index.html?ui=P_0x31e6d5a90_3&reconnect=auto" class="pyvista…

In [10]:
## Calculate heat flux, evaluate at surface — proxy for boundary layer thickness

In [11]:
t_soln.stats()

(12466,
 0.49986938399552316,
 -1.6180578938125931,
 2.70905145440194,
 6231.371740888192,
 64.81584943734697,
 0.5805206270949029)

In [12]:
0/0

ZeroDivisionError: division by zero

In [ ]:
flux_solver = uw.systems.Projection(mesh, flux)

# Conductive flux only !
radial_flux = -mesh.vector.gradient(t_soln.sym[0]).dot(mesh.CoordinateSystem.unit_e_0)
radial_flux *= sympy.exp(-100*(r-1)**2) 

flux_solver.uw_function = radial_flux
flux_solver.smoothing = 1.0e-3
flux_solver.solve()

In [ ]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pvmesh = vis.mesh_to_pv_mesh(mesh)
pvmesh.point_data["dTdz"] = vis.scalar_fn_to_pv_points(pvmesh, flux.sym[0])
pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
pvmesh.point_data["V"] -= pvmesh.point_data["V"].mean()

velocity_points = vis.meshVariable_to_pv_cloud(v_soln)
velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, v_soln.sym)

# point sources at cell centres
skip = 8
points = np.zeros((mesh._centroids[::skip].shape[0], 3))
points[:, 0] = mesh._centroids[::skip, 0]
points[:, 1] = mesh._centroids[::skip, 1]
point_cloud = pv.PolyData(points)

pvstream = pvmesh.streamlines_from_source(
    point_cloud,
    vectors="V",
    integration_direction="both",
    max_time=0.5,
)

pl = pv.Plotter(window_size=[1000, 1000])

pl.add_mesh(
    pvmesh,
    cmap="Reds",
    edge_color="Black",
    show_edges=True,
    scalars="dTdz",
    use_transparency=False,
    opacity=1.0,
)

# 
# pl.add_mesh(pvmesh, "Black", "wireframe",  opacity=0.1)

# pl.add_points(point_cloud, color="White", point_size=3.0, opacity=0.25)

# pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=0.0001, show_scalar_bar=True)
pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)
    
# pl.remove_scalar_bar("V")

imagefile = os.path.join(image_dir, checkpoint_base) + f"_dTdr_{step}.png"

pl.screenshot(filename=imagefile, window_size=(750, 750), return_img=False)
# OR


In [ ]:
pl.show()